In [1]:
from bs4 import BeautifulSoup
import json
import re
import urllib2
import datetime
from os import walk

### Example of URL we have to find:
https://en.wikipedia.org/wiki/Talk:Antifa_(United_States)/Archive_4#Request_for_comment

### The URL we will use:

https://en.wikipedia.org/w/index.php?search=RfC%3A_Should_%7B%7Binfobox_criminal%7D%7D_be_used_in_this_article%3F+prefix%3ATalk%3AAloysius+Stepinac&title=Special:Search&profile=default&fulltext=1&searchToken=mhp1jccsq7yxe23x5n6z1r2w

In [3]:
# get the file names
diff_folder_dir = "/Users/jane/rfc-analysis/diffs/"
diff_files = []
for (dirpath, dirnames, filenames) in walk(diff_folder_dir):
    diff_files.extend(filenames)

In [60]:
def find_rfc_url(rfc_info):
    rfc_anchor = rfc_info["parsed_rfc_title"]
    rfc_page_title = rfc_info["page_title"]
    
    rfc_url_candidates = find_rfc_url_from_archive(rfc_anchor, rfc_page_title)
    return rfc_url_candidates

In [58]:
def find_rfc_url_from_archive(rfc_anchor, rfc_page_title):
    """ make sure to check for duplicates """
    url_candidates = get_rfc_url_candidates(rfc_anchor, rfc_page_title)
    narrowed_url_candidates = set()
    print url_candidates
    if len(url_candidates) > 0:
        # make sure to cover cases when there are multiple candidates
        for c in url_candidates:
            anchor_section = c.split("#")[-1]
            if anchor_section.strip() == rfc_anchor.strip():
                # include only the exact same ones
                narrowed_url_candidates.add(c.strip())
                return list(narrowed_url_candidates)

In [57]:
def get_rfc_url_candidates(rfc_anchor, rfc_page_title):
    archive_search_url = "https://en.wikipedia.org/w/index.php?search=" + rfc_anchor.strip() + "&prefix=" + rfc_page_title + "&title=Special:Search&profile=default&fulltext=1"
    archive_search_url = archive_search_url.replace(" ", "_").encode('utf-8') # have to do this because of the page title
#     print 'archive search url: ' + archive_search_url
    archive_search_page = urllib2.urlopen(archive_search_url)
    soup = BeautifulSoup(archive_search_page, 'lxml')
    try:
#         candidates = [c.find(class_='searchalttitle').a['href'].strip() for c in soup.find_all(class_='mw-search-result-heading')]
        candidates = []
        for c in soup.find_all(class_='mw-search-result-heading'):
            if c.find(class_='searchalttitle') is not None:
                candidates.append(c.find(class_='searchalttitle').a['href'].strip())
        return candidates
    except Exception:
        return []

function that iterates over the files and tries to find the corresponding url

In [182]:
def find_batch_urls(rfc_files, folder_dir, single_url_found, multiple_url_found, url_unfound, file_name_to_replace):
    for i, file_name in enumerate(rfc_files):
        if ".json" in file_name:
#             print file_name
            rfc_id = file_name.replace(file_name_to_replace, "").replace(".json", "")
            if rfc_id not in single_url_found.keys() and rfc_id not in multiple_url_found.keys() and rfc_id not in url_unfound:
                path = folder_dir + file_name
                print rfc_id
                with open(path) as file:
                    rfc_info = json.load(file)
                if "parsed_rfc_title" in rfc_info:
                    print str(i) + ' - ' + rfc_info["id"]
                    rfc_id = rfc_info["id"]
                    try:
                        urls = find_rfc_url(rfc_info)
                        if urls is None:
                            url_unfound.add(rfc_id)
                        else:
                            if len(urls) > 1:
                                multiple_urls_found[rfc_id] = urls
                            else:
                                single_url_found[rfc_id] = urls[0]
                    except urllib2.HTTPError:
                        url_unfound.add(rfc_id)
    return single_url_found, multiple_url_found, url_unfound

### DIFFS

In [163]:
single_url_found, multiple_url_found, url_unfound = find_batch_urls(diff_files, diff_folder_dir, single_url_found, multiple_url_found, url_unfound, "diff_added_")

426415586
426416144
429210861
429210876
429210903
429210939
429210951
429210964
429211028
429211048
429211061
429211080
429211140
429211156
429211163
429211171
429211207
429211212
429211226
429211245
429211274
429211285
429211289
429211352
429211357
429211361
429211370
429211378
429211394
429211402
429211440
429211492
429211498
429211505
429211514
429211587
429211611
429211619
429211627
429211669
429211728
429211741
429211800
429211831
429211838
429211852
429211864
429211876
429211883
429211908
429211926
429211937
429211967
429211989
429212004
429213183
429213197
429278348
429433581
429460060
429467878
429505797
429529527
429594471
429603746
429612677
429629849
429689409
429689429
429726026
429741104
429749213
429763928
430069435
430160945
430208910
430225313
430242386
430360160
430360185
430505448
430567589
430592781
430780104
430780126
430827142
430841597
430900130
430904350
431052421
431086041
431192127
431584376
431715112
431743666
431785491
431853880
431853901
431862947
431863039


['/wiki/Talk:Aloysius_Stepinac#RfC:_Should_{{infobox_criminal}}_be_used_in_this_article?']
684530725
6211 - 684530725
["/wiki/Talk:Volleyball_at_the_2014_Asian_Games#User:Mohsen1248's_dogmatic_approach"]
684589423
6212 - 684589423
['/wiki/Wikipedia:Village_pump_(policy)/Archive_123#RfC_-_should_we_allow_primary_sources_sent_in_to_OTRS']
684607487
6213 - 684607487
["/wiki/Talk:Yugoslavia/Archive_3#Should_FR_Yugoslavia's_image_be_included_in_the_lead"]
684642724
6214 - 684642724
['/wiki/Talk:Elan_Carr#RfC:_The_endorsements_make_this_an_advertisement_page']
684649496
6215 - 684649496
['/wiki/Talk:Superpower/Archive_12#RfC:_Ottoman_Empire__superpower']
684673485
6216 - 684673485
['/wiki/Talk:List_of_Walt_Disney_Pictures_films#Walt_Disney_Pictures_India']
684707015
6217 - 684707015
[]
684719261
6218 - 684719261
['/wiki/Talk:Exo%27s_Showtime#Merger_proposal']
684793570
6219 - 684793570
['/wiki/Talk:Hillary_Clinton/Archive_27#rfc_-_quinnipiac_"liar"_poll']
684828135
6220 - 684828135
['/wiki/W

['/wiki/Talk:Intifada#RfC:_on_the_Third_Intifada']
687721675
687739181
6295 - 687739181
['/wiki/Talk:Peter_of_Spain#Rfc']
687841091
6296 - 687841091
['/wiki/Talk:Fawaz_Gerges#RFC_-_Promotional_Commentary_in_Header']
687853996
6297 - 687853996
['/wiki/Talk:Gary_Cooper/Archive_1#Recent_edits']
687869955
6298 - 687869955
['/wiki/Talk:Schizoaffective_disorder#Request_for_comment_on_the_painting(s)']
687876784
6299 - 687876784
[u"/wiki/Talk:Alternative_theories_of_the_location_of_Great_Moravia#Juraj_Sklen\xe1r's_view"]
687881993
6300 - 687881993
['/wiki/Talk:Bill_Cosby/Archive_4#Discussion:_Should_the_lead_sentence_mention_the_sexual_assault_accusations?']
687895953
6301 - 687895953
['/wiki/Talk:Hello_(Adele_song)#RfC:_Including_official_video_links_in_external_links']
687945421
6302 - 687945421
['/wiki/Talk:War_in_Donbass/Archive_9#RfC:_Combatants']
688106994
6303 - 688106994
['/wiki/Talk:Jewish_Israeli_stone_throwing#RfC_from_Palestinian_stone-throwing']
688129625
6304 - 688129625
['/wiki

['/wiki/Template_talk:Hindu_philosophy#Does_other_religions_should_be_included_in_Hindu_philosophy_template?']
690354832
6367 - 690354832
['/wiki/Talk:Shaun_the_Sheep_Movie#Request_for_comment']
690363103
6368 - 690363103
[]
690405990
6369 - 690405990
['/wiki/Talk:Cold_War_II/Archive_2#Move_portions_to_other_pages?']
690411407
6370 - 690411407
['/wiki/Talk:Fa_La_La_(album)#RfC:_should_the_article_Fa_La_La_(album)_be_converted_into_a_redirect_to_the_artist?']
690436017
690478059
6372 - 690478059
['/wiki/Talk:Royal_Households_of_the_United_Kingdom#Possible_hoax_or_hoaxes']
690507333
6373 - 690507333
['/wiki/Talk:Kuwait_Airways#RFC:_Should_a_threat_of_legal_action_by_the_Secretary_of_Transportation_against_the_airline_be_included_in_the_article?']
690561716
6374 - 690561716
['/wiki/Talk:Atlantis/Archive_7#Rfc:_Atlantis_as_Fiction,_Allegory']
690566186
6375 - 690566186
['/wiki/Talk:Kim_Jong-un/Archive_5#Rfc:_Use_of_non-free_image']
690605354
6376 - 690605354
['/wiki/Talk:Greco-Italian_War/

['/wiki/Talk:Mariah_Carey/Archive_13#RfC:_Are_"African_American"_categories_supported_by_sources_and_policy?']
692889009
692937381
6443 - 692937381
['/wiki/Template_talk:Infobox_language/Archive_8#RfC:_What_should_the_language_infobox_display_when_editors_have_not_found_any_speaker_figures?']
692942546
6444 - 692942546
['/wiki/Talk:The_Game_discography#Album_sales_figures']
692965211
6445 - 692965211
['/wiki/Talk:Monarchy_of_Ceylon#RfC:_Merge_and_disambiguate']
693000688
6446 - 693000688
[]
693008711
693031735
6448 - 693031735
['/wiki/Talk:List_of_Power_Rangers_Dino_Charge_characters#RfC:_Appropriate_detail_of_plot_summary_information']
693039227
6449 - 693039227
['/wiki/Talk:Ivo_Andri%C4%87/Archive_2#Name,_family']
693139693
6450 - 693139693
['/wiki/Talk:American_Broadcasting_Company/Archive_1#Propose_moratorium']
693190138
6451 - 693190138
['/wiki/Wikipedia_talk:Deletion_policy/Archive_46#RFC:_delete_and_redirect']
693190142
6452 - 693190142
[]
693254299
6453 - 693254299
['/wiki/Talk

...

['/wiki/Talk:List_of_autistic_fictional_characters#RFC']
804115230
10098 - 804115230
['/wiki/Talk:Asahi-ekimae-d%C5%8Dri_Station#Tram_stop_and_train_station_policy']
804121884
10099 - 804121884
['/wiki/Talk:Spanish_Empire#RfC_about_the_infobox_Spanish_Empire_image']
804163722
10100 - 804163722
[]
804177523
10101 - 804177523
['/wiki/Talk:Red#Citations_in_lead']
804191525
10102 - 804191525
['/wiki/Wikipedia_talk:Consensus/Archive_20#Request_for_Comment_-_add_subsection_to_WP:Local_consensus_about_consensus_levels']
804216901
10103 - 804216901
['/wiki/Wikipedia:Village_pump_(policy)/Archive_138#Anti/Pro-Trump/other_politician_user_categories_and_userboxes?']
804233706
10104 - 804233706
['/wiki/Wikipedia_talk:WikiProject_Football/Archive_112#RfC_at_Talk:Kwadwo_Poku']
804233715
804233724
804271126
10107 - 804271126
['/wiki/Talk:The_Gifted_(TV_series)#RfC']
804354860
10108 - 804354860
['/wiki/Talk:Iyoki_Station#Tram_stop_and_train_station_policy']
804382333
804397067
10110 - 804397067
['/wik

["/wiki/Talk:Julie_Payette#RfC_on_subject's_title"]
806357372
10180 - 806357372
['/wiki/Talk:Limes#Rfc_regarding_Limes']
806357378
10181 - 806357378
['/wiki/Talk:White_power#Request_for_Comment']
806456080
10182 - 806456080
['/wiki/Wikipedia_talk:WikiProject_Television_Stations/Archive_10#Television_channel_lists', '/wiki/Wikipedia_talk:WikiProject_Television_Stations/Archive_9#Television_channels_Wiki', '/wiki/Wikipedia_talk:WikiProject_Television/Archive_25#Television_channels', '/wiki/Wikipedia_talk:WikiProject_Television/Archive_20#Template:Infobox_television_channel_parameter_change', '/wiki/Wikipedia_talk:WikiProject_Television/Archive_12#List_of_Kurdish_television_channels', '/wiki/Wikipedia_talk:WikiProject_Television_Stations/Archive_7#Category:Defunct_television_channels_and_Category:Defunct_television_networks', '/wiki/Wikipedia_talk:WikiProject_Television/Archive_16#China_Central_Television_channels', '/wiki/Wikipedia_talk:WikiProject_Television/Archive_17#Television_channe

['/wiki/Talk:Ed_Gillespie#Request_for_comment_on_Latino_Victory_Fund_ad']
808632017
10246 - 808632017
['/wiki/Talk:Rihanna#RfC']
808670988
10247 - 808670988
['/wiki/Talk:Monero_(cryptocurrency)#Request_for_comment_on_content_and_source_quality']
808745907
10248 - 808745907
['/wiki/Wikipedia:Village_pump_(policy)#Consensus_and_copyright_law']
808767233
10249 - 808767233
['/wiki/Talk:Presidential_Advisory_Commission_on_Election_Integrity#RfC:_Meeting_subsections']
808842363
10250 - 808842363
['/wiki/Talk:Antifa_(United_States)/Archive_4#Request_for_comment']
808938124
10251 - 808938124
['/wiki/Talk:June_2017_Brussels_attack#RfC__on_MILAN_SCHREUER_and_DAN_BILEFSKY_article_in_New_York_Times']
808954239
10252 - 808954239
['/wiki/Wikipedia_talk:Manual_of_Style/Film#RFC_for_changing_the_production_section_guidelines._2']
808969655
808985960
10254 - 808985960
['/wiki/Talk:His_Dark_Materials#Request_for_comment:_should_we_keep_the_Terminology_and_Pronunciation_sections?']
809050547
809096591
10

['/wiki/Talk:List_of_areas_of_London#RFC_Project_to_cleanup_the_child_pages_of_this_page']
811788668
10326 - 811788668
['/wiki/Talk:Ireland#"British_Isles"_RfC', '/wiki/Talk:Ireland/Archive_9#RFC:_Talk:Irish_language#Order_of_the_First_Paragraph', '/wiki/Talk:Ireland/Archive_13#British_Isles', '/wiki/Talk:Ireland/Archive_7#Title_for_{{British_Isles}}_template', '/wiki/Talk:Ireland/Archive_14#"So_called"_British_Isles', '/wiki/Talk:Ireland/Archive_11#Special_Rfc_proposal', '/wiki/Talk:Ireland_(disambiguation)#RfC:_controversial_multi-page_move']
811788674
10327 - 811788674
['/wiki/Talk:Monster_House_(video_game)#Request_for_Comment']
811867644
10328 - 811867644
['/wiki/Wikipedia_talk:WikiProject_Cricket#Request_for_comment_on_achievements_and_awards_boxes']
811880771
10329 - 811880771
['/wiki/Talk:Poutine#RfC_about_cuisine_classification']
811895147
10330 - 811895147
['/wiki/Talk:New_Albion#Request_for_Comment_Re:_Fringe_Theories_Section']
812072772
10331 - 812072772
['/wiki/Wikipedia_t

['/wiki/Talk:Matt_Lauer#Requests_for_comment_from_wider_community']
815144870
10409 - 815144870
['/wiki/Template_talk:Yesno#Support_on/off_detection']
815144874
10410 - 815144874
['/wiki/Talk:United_States_recognition_of_Jerusalem_as_Israeli_capital#RfC_for_130_Jewish_studies_scholars']
815167662
10411 - 815167662
['/wiki/Talk:List_of_sovereign_states#RfC:_Including_territories_with_contested_statehood_to_list_of_sovereign_states']
815262830
815262847
10413 - 815262847
['/wiki/Talk:Guonei#RFC_on_Move_Request']
815262852
10414 - 815262852
['/wiki/Talk:Wandu#RFC_on_Move_Request']
815278090
10415 - 815278090
['/wiki/Talk:Spire_Credit_Union#Alleged-acronym_RfC']
815299287
10416 - 815299287
[]
815299290
10417 - 815299290
[]
815299296
10418 - 815299296
[]
815299301
10419 - 815299301
[]
815357320
10420 - 815357320
['/wiki/Talk:List_of_starting_quarterbacks_in_the_National_Football_League#Add_ages_of_QBs_to_table?']
815357325
10421 - 815357325
['/wiki/Talk:The_Bible_and_violence#Rfc']
81539491

### Results:
#### There were no issues due to same section title. 
#### In total 6,712 new urls were found from stage 1

In [166]:
len(set(single_url_found.values()))

6712

In [183]:
len(set(multiple_url_found.values()))

0

In [170]:
with open("/Users/jane/rfc-analysis/diff_url_single.json", "w") as file:
    json.dump(single_url_found, file)

In [171]:
with open("/Users/jane/rfc-analysis/content_single_found_batch.json") as file:
    content_found = json.load(file)

In [178]:
len(set(content_found.values()))

710

In [173]:
all_found = {}

In [174]:
for id, url in content_found.items():
    all_found[id] = url

In [175]:
for id, url in single_url_found.items():
    all_found[id] = url

In [180]:
len(set(all_found.values()))

7295

In [179]:
with open("/Users/jane/rfc-analysis/all_found_url.json", "w") as file:
    json.dump(all_found, file)